<a href="https://colab.research.google.com/github/tavishcode/fyp/blob/master/ml_caching_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input, Activation, CuDNNGRU, GRU, Dense
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import scale, StandardScaler, RobustScaler
from collections import OrderedDict, defaultdict
from google.colab import drive
import time
drive.mount('/content/drive')
plt.rcParams['figure.figsize'] = [10, 8]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
train = np.load('drive/My Drive/train_set.npy')
train.shape

(27222, 100, 9)

### Simple GRU

In [13]:
window = 7
num_features = 9
model = Sequential()
model.add(CuDNNGRU(64, input_shape=(window, num_features)))
model.add(Dense(1))
model.add(Activation('relu'))
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnngru_5 (CuDNNGRU)       (None, 64)                14400     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0         
Total params: 14,465
Trainable params: 14,465
Non-trainable params: 0
_________________________________________________________________


In [0]:
preds = np.zeros((train.shape[0], train.shape[1]))
for w in range(window):
  preds[:, w] = 0
for i in range(window, train.shape[1]):
  x = np.copy(train[:, i - window : i, :]).reshape(-1, window, 9)
  y = np.copy(train[:, i, 0]).reshape(-1, 1)
  p = model.predict(x).flatten()
  preds[:, i] = p
  start = time.clock()
  model.fit(x, y, epochs=50, verbose=0)
  print(time.clock() - start)
  print(str(i)+'/'+str(train.shape[1]))
model.save('simple_gru.h5')

Instructions for updating:
Use tf.cast instead.
279.148273
7/100
278.792569
8/100
279.37132600000007
9/100
279.419991
10/100
279.31816000000003
11/100
279.560017
12/100
278.8045609999999
13/100
278.41947900000014
15/100
278.1631419999999
16/100
277.95934099999977
17/100
280.155655
18/100
280.785981
19/100
278.6217489999999
20/100
278.3911750000002
21/100
278.9129300000004
22/100
279.2308069999999
23/100
279.4474279999995
24/100
280.0068000000001
25/100
279.62146200000007
26/100
279.40147499999966
27/100
278.7786150000002
28/100
